In [ ]:
import getpass
from dalle.imperative.api.labs import get_bearer_token

print("Attempting to get token for DALL-E...")
token = get_bearer_token(
    input("OpenAI Username: "),
    getpass.getpass('OpenAI Password: ')
)
print("Token:", token[:5] + ("*" * (len(token) - 5)))

In [ ]:
from dalle.imperative.api.labs import get_tasks

from_ts = 0
print("Getting tasks for tasks starting from timestamp", from_ts)
tasks = get_tasks(token, from_ts)
print("# of Tasks:", len(tasks.data))

In [ ]:
import matplotlib.pyplot as plt
from PIL.Image import Image
import textwrap, os

def display_images(
    images: [Image],
    columns=4, width=20, height=8, max_images=24,
    label_wrap_length=50, label_font_size=8,
    disable_axis=True):

    if not images:
        print("No images to display.")
        return

    if len(images) > max_images:
        print(f"Showing {max_images} images of {len(images)}:")
        images=images[0:max_images]

    height = max(height, int(len(images)/columns) * height)
    plt.figure(figsize=(width, height))
    for i, image in enumerate(images):

        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        if disable_axis:
            plt.axis('off')
        plt.imshow(image)

        if hasattr(image, 'filename'):
            title=image.filename
            if title.endswith("/"): title = title[0:-1]
            title=os.path.basename(title)
            title=textwrap.wrap(title, label_wrap_length)
            title="\n".join(title)
            plt.title(title, fontsize=label_font_size)

In [ ]:
from urllib.request import urlopen
from PIL import Image

images = []
for task in tasks.data:
    for generation in task.generations:
        img = urlopen(generation.generation.image_path)
        images.append(Image.open(img))
    if len(images) >= 4:
        break

display_images(images)

In [ ]:
from dalle.imperative.api.labs import create_text2im_task, poll_for_task_completion

print("Attempting to create task...")
pending_task = create_text2im_task(token, "A cute cat")
print(pending_task.id, pending_task.status, pending_task.generations)
print("Waiting for task to complete...")
task = poll_for_task_completion(token, pending_task.id)
print(task.id, task.status, task.generations)
display_images([Image.open(urlopen(x.generation.image_path)) for x in task.generations])